In [1]:
from datasets import load_dataset

dataset = load_dataset("autopilot-ai/Gujarati-Grammarly-Datasets",data_files=["sentence-pairs/gujarati_sentences_1M.csv"], split = "train[100000:110000]")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset csv (/root/.cache/huggingface/datasets/autopilot-ai___csv/autopilot-ai--Gujarati-Grammarly-Datasets-7432f8bd995d09c0/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


In [2]:
dataset

Dataset({
    features: ['Correct', 'Incorrect'],
    num_rows: 10000
})

In [3]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

checkpoint = "Jayveersinh-Raj/guj-grammar-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda115.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 115
CUDA SETUP: Loading binary /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda115.so...


/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)


In [4]:
model.to("cuda")

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

In [6]:
import torch
# Input sentence for prediction
input_sentence = "શિયાળાની ઠંડી હોય તો મોટાભાગના લોકોને શરદી, ઉધરસ અને છીંક આવવાની ટકલિફ હૌય છે."

# Tokenize the input sentence
input_ids = tokenizer.encode(input_sentence, return_tensors="pt").to("cuda")

# Generate predictions
with torch.no_grad():
    output_ids = model.generate(input_ids, max_new_tokens=200)

# Decode the generated output
output_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print the generated output
print("Input Sentence:", input_sentence)
print("Generated Sentence:", output_sentence)

Input Sentence: શિયાળાની ઠંડી હોય તો મોટાભાગના લોકોને શરદી, ઉધરસ અને છીંક આવવાની ટકલિફ હૌય છે.
Generated Sentence: શિયાળાની ઠંડી હોય તો મોટાભાગના લોકોને શરદી, ઉધરસ અને છીંક આવવાની તકલીફ હોય છે.


In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Create lists to store input, ground truth, and generated sentences
input_sentences = []
ground_truth_sentences = []
generated_sentences = []

# Iterate through the dataset
for example in tqdm(dataset, desc="Generating predictions"):
    # Assuming 'Incorrect' is the key for incorrect sentences
    input_sentence = example['Incorrect']
    
    # Tokenize the input sentence
    input_ids = tokenizer.encode(input_sentence, return_tensors="pt").to("cuda")

    # Generate predictions
    with torch.no_grad():
        output_ids = model.generate(input_ids, max_new_tokens=200)

    # Decode the generated output
    generated_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Append to lists
    input_sentences.append(input_sentence)
    ground_truth_sentences.append(example['Correct'])
    generated_sentences.append(generated_sentence)

# Create a DataFrame
df = pd.DataFrame({
    'Input Sentence': input_sentences,
    'Ground Truth': ground_truth_sentences,
    'Generated Sentence': generated_sentences
})

# Save the DataFrame to a CSV file
df.to_csv('predictions_base.csv', index=False)

# Display the DataFrame
print(df.head())

Generating predictions: 100%|█████████▉| 9985/10000 [2:12:03<00:12,  1.22it/s]  

In [26]:
df

,Input Sentence,Ground Truth,Generated Sentence
0,૯મી ઓઘશત ૨૦૧૬ના રૌજ આદિવાસી વિકાસ સંગઠન દ્વારા...,૯મી ઓગસ્ટ ૨૦૧૬ના રોજ આદિવાસી વિકાસ સંગઠન દ્વાર...,૯મી ઓગસ્ટ ૨૦૧૬ના રોજ આદિવાસી વિકાસ સંગઠન દ્વાર...
1,"આ પતાવટની આંતરમાળખા ખૂબ સારી રીતે વિકસિત નથી, ...","આ પતાવટની આંતરમાળખા ખૂબ સારી રીતે વિકસિત નથી, ...","આ પતાવટની આંતરમાળખા ખૂબ સારી રીતે વિકસિત નથી, ..."
2,"વહીવટ બીલડીઁઘ નજીક પાછળના બાજુ પર, હોટેલ આંતરિ...","વહીવટ બિલ્ડિંગ નજીક પાછળના બાજુ પર, હોટેલ આંતર...","વહીવટ બિલ્ડિંગ નજીક પાછળના બાજુ પર, હોટેલ આંતર..."
3,ઘૂરૂવારૈ શવારૈ બેંકો ખુલતા પહૈલા પ્રતિબંધિત નો...,ગુરુવારે સવારે બેંકો ખુલતા પહેલા પ્રતિબંધિત નો...,ગુરુવારે સવારે બેંકો ખુલતા પહેલા પ્રતિબંધિત નો...
4,ઈન્ડિયન આઈડલ 11ના આગામી એપીશૌડમાઁ ઉદિત નારાયણ ...,ઈન્ડિયન આઈડલ 11ના આગામી એપિસોડમાં ઉદિત નારાયણ ...,ઈન્ડિયન આઈડલ 11ના આગામી એપીસોડમાં ઉદિત નારાયણ ...
...,...,...,...
995,અહીં તમને અચરજ પમાડે એવું ઘણું બધું જૌવા મળી ર...,અહીં તમને અચરજ પમાડે એવું ઘણું બધું જોવા મળી ર...,અહીં તમને અચરજ પમાડે એવું ઘણું બધું જોવા મળી ર...
996,આ રોગવિજ્ઞાન વિકાસ તરફ દોરી જાય છે.,આ રોગવિજ્ઞાન વિકાસ તરફ દોરી જાય છે.,આ રોગવિજ્ઞાન વિકાસ તરફ દોરી જાય છે.
997,તેમણે 35 હતી.,તેમણે 35 હતી.,તેમણે 35 હતી.
998,તેમાં કહેવામાં આવયૂઁ કૈ ડેમોક્રેટ્‌સે યૂકરૈનનિ...,તેમાં કહેવામાં આવ્યું કે ડેમોક્રેટ્‌સે યુક્રેન...,તેમાં કહેવામાં આવ્યું કે ડેમોક્રેટ્ સે યુક્રેન...


In [17]:
!pip install numba

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 3.7 MB 851 kB/s eta 0:00:01
     |████████████████████████████████| 43.6 MB 9.8 kB/s  eta 0:00:01


In [19]:
from numba import cuda
cuda.select_device(1)
cuda.close()
cuda.select_device(1)

<weakproxy at 0x7f39ab1b2f40 to Device at 0x7f39ab1b07f0>